In [1]:
from utils import load_data, check_gpu, check_dir
from clean_session import preprocess_sessions
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
import matplotlib.pyplot as plt
from clean_session import preprocess_sessions
from create_model_inputs import prepare_data, compute_session_fts, create_model_inputs
import datetime
%matplotlib inline
%load_ext autoreload
%autoreload 2
def fprint(df, name):
    print(f'{name} shape: ({df.shape[0]:,}, {df.shape[1]})')
    

In [2]:
nrows = 1000000
df = prepare_data('train', convert_action_type=False, nrows=nrows, recompute=True)
logger.info('Compute session features')
df = compute_session_fts(df, 'train')

[05-14 09:35:47 - utils-106 - load_data - INFO] Loading train using 1,000,000 rows (999,995 trimmed) which is 6.28% out of total train data


In [4]:
df = create_model_inputs('train', inspection=True, nrows=100000, recompute=True)

[05-14 09:36:24 - create_model_inputs-194 - create_model_inputs - INFO] 
========== Creating TRAIN model inputs with 100,000 rows and recompute=True ==========
[05-14 09:36:24 - create_model_inputs-205 - create_model_inputs - INFO] Prepare train data
[05-14 09:36:24 - utils-106 - load_data - INFO] Loading train using 100,000 rows (99,970 trimmed) which is 0.63% out of total train data
[05-14 09:36:24 - create_model_inputs-17 - flogger - INFO] raw train shape: (99,970, 12)
[05-14 09:36:24 - clean_session-60 - preprocess_sessions - INFO] Dropping duplicates
[05-14 09:36:24 - clean_session-18 - remove_duplicates - INFO] Before dropping duplicates df shape: (99,970, 12)
[05-14 09:36:24 - clean_session-21 - remove_duplicates - INFO] After dropping duplicates df shape: (40,977, 12)
[05-14 09:36:24 - clean_session-62 - preprocess_sessions - INFO] Cliping session dataframe up to last click out (if there is clickout)
[05-14 09:36:26 - clean_session-65 - preprocess_sessions - INFO] Filtering out

### comapre session with size 1 and the rest

In [5]:
df.head()

,session_id,user_id,timestamp,step,action_type,current_filters,reference,impressions,prices,timestamp_session_duration,timestamp_dwell_time_prior_clickout,current_filters_last_filters,session_id_size,reference_last_reference_id,cfs,time_steps,target,last_ref_ind
0,0008f9d105bbf,0J9R30BACDXZ,2018-11-04 22:07:13,2,clickout item,NaN,1427147,"[8594588, 77720, 891699, 9903484, 10130396, 31...","[6.672032945461067, 6.401917196727186, 6.19440...",32.0,3.496508,NaN,0.693147,"Myrtle Beach, USA",NaN,25,6,NaN
1,0018c0eba0994,L7WN0SC4FCCN,2018-11-03 22:45:12,1,clickout item,NaN,54922,"[7810226, 1284528, 324226, 1473939, 2297996, 9...","[5.056245805348308, 5.0238805208462765, 4.8441...",NaN,NaN,NaN,0.000000,NaN,NaN,25,22,NaN
2,002619d05064b,2ZT0V5495OQ4,2018-11-06 12:30:32,3,clickout item,NaN,4516564,"[4683754, 2651328, 1360468, 4516564, 5754526, ...","[4.23410650459726, 4.060443010546419, 4.110873...",81.0,4.025352,NaN,1.098612,4983898,NaN,13,3,0.44
3,003c9a4b892fa,APY367QNO5LY,2018-11-02 12:57:43,2,clickout item,NaN,9421486,"[6542484, 4217988, 7280316, 5513842, 8576928, ...","[5.497168225293202, 5.181783550292085, 5.37063...",60.0,4.110874,NaN,0.693147,"Ubatuba, Brazil",NaN,12,11,NaN
4,00408fc89c60a,3H8BQNK8U77L,2018-11-04 23:27:29,3,clickout item,NaN,6333948,"[1056384, 106412, 1656477, 2392526, 2684345, 5...","[6.300785794663244, 4.941642422609304, 5.03043...",7.0,1.945910,NaN,1.098612,1656947,NaN,24,6,0.32


In [6]:
ones = df[df['session_id_size']==1].reset_index(drop=True)
multiples = df[df['session_id_size']>1].reset_index(drop=True)

In [7]:
oids = ones['session_id'].unique()
mids = multiples['session_id'].unique()

In [ ]:
# random select ids to look at 